In [2]:
#Glosario de imports
from bs4 import BeautifulSoup
import requests
import re
import os
from datetime import datetime
import firebase_admin
from firebase_admin import credentials, firestore

In [4]:
cred = credentials.Certificate("rich12-efbe7-firebase-adminsdk-yeszr-7260cd4be1.json")
firebase_admin.initialize_app(cred)

In [22]:
#Metodos sacar URL's de cada medio

def lista_urls_elmundo(url):
    lista = [] 
    html = requests.get(url)

    web = BeautifulSoup(html.text, 'html.parser')
    noticias = web.find_all("a", class_="ue-c-cover-content__link-whole-content")
    
    for url in noticias:
        url_noticia = url.get('href')
        lista.append(url_noticia)
    return lista




In [28]:
lista_ciencia_elmundo = lista_urls_elmundo("https://www.elmundo.es/ciencia-y-salud/ciencia.html")
lista_tecnologia_elmundo = lista_urls_elmundo("https://www.elmundo.es/tecnologia.html")
lista_salud_elmundo = lista_urls_elmundo("https://www.elmundo.es/ciencia-y-salud/salud.html")

In [24]:
db = firestore.client()

In [29]:
def guardar_noticias_mundo(lista_urls, tipo):
    index = 0
    for url in lista_urls:
        html = requests.get(url)
        web = BeautifulSoup(html.text, 'html.parser')
        
        etiquetas = ""
        tags = web.find_all('li', class_="ue-c-article__tags-item")
        titulo = web.find('h1', class_="ue-c-article__headline js-headline")
        entradilla = web.find('p', class_="ue-c-article__standfirst")

        if titulo == None:
            titulo = ""
        else:
            titulo = titulo.text

        if entradilla == None:
            entradilla = ""
        else:
            entradilla = entradilla.text
        if tags == None:
            tags = ""
        else:
            for t in tags:
                etiquetas = etiquetas + ", " + t.get_text()

        cuerpo= ""
        for para in web.find_all('p', class_=""):
            cuerpo = cuerpo + para.get_text()
        
        
        index = index + 1
        collection = db.collection('no_odio')
        
        data = [({"titulo": titulo,
           "entradilla": entradilla,
           "cuerpo": cuerpo})]
        
        documento = 'noticia' + str(tipo) + str(index)
        collection.document(documento).set(data[0])
        
        


In [30]:
guardar_noticias_mundo(lista_tecnologia_elmundo, "tecnologia")
guardar_noticias_mundo(lista_ciencia_elmundo, "ciencia")
guardar_noticias_mundo(lista_salud_elmundo, "salud")
